

- A **tensor** is a generalization of vectors and matrices to potentially higher dimensions.
- Internally, TensorFlow represents tensors as **n-dimensional arrays of base datatypes**.
- A `tf.Tensor` object represents *a partially defined* computation that will *eventually* produce a value.
- A tf.Tensor has the following properties:
    - a data type (float32, int32, or string, for example)
    - a shape
- Each element in the Tensor has the same data type, and the data type is always known. The shape (that is, the number of dimensions it has and the size of each dimension) *might be only partially known*. In some cases it's only possible to find the shape of a tensor at graph execution time
- The main tensor types are:
    - `tf.Variable`
    - `tf.Constant`
    - `tf.Placeholder`
    - `tf.SparseTensor`
- With *the exception of tf.Variable*, the value of a tensor is **immutable**, which means that in the context of a single execution tensors only have a single value. However, evaluating the same tensor twice can return different values; for example that tensor can be the result of reading data from disk, or generating a random number.

### Rank

- The **rank** of a tf.Tensor object is its number of dimensions. Synonyms for rank include *order* or *degree* or *n-dimension*.
- Rank 0
    - Scalar (magnitude only)
    - A string is treated as a single item in TensorFlow, not as a sequence of characters
```python
mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable((12.3, -4.85), tf.complex64)
```
- Rank 1
    - Vector (magnitude and direction)
```python
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers  = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([(12.3, -4.85), (7.5, -6.23)], tf.complex64)
```
- Rank 2
    - 	Matrix (table of numbers)
- Rank 3
    - 3-Tensor (cube of numbers)
- Rank n
    - n-Tensor (you get the idea)
```python
my_image = tf.zeros([10, 299, 299, 3])  # batch x height x width x color
```

In [11]:
my_image = tf.zeros([10, 299, 299, 3])  # batch x height x width x color
sess = tf.Session()

my_image, tf.rank(my_image), my_image[1, 2, 3, 1]

(<tf.Tensor 'zeros_6:0' shape=(10, 299, 299, 3) dtype=float32>,
 <tf.Tensor 'Rank_8:0' shape=() dtype=int32>,
 <tf.Tensor 'strided_slice_3:0' shape=() dtype=float32>)

In [ ]:
sess.run([tf.rank(my_image), my_image[1, 2, 3, 1]])

- `tf.Tensor.shape` property: reading the shape property of a tf.Tensor object. This method returns a TensorShape object, which is a convenient way of representing partially-specified shapes
- `tf.shape` operation: get a tf.Tensor that will represent the fully-defined shape of another tf.Tensor at runtime

In [15]:
 my_image.get_shape(), my_image.shape
zeros = tf.zeros(tf.shape(my_image))

In [18]:
tf.shape(my_image), my_image.get_shape(), my_image.shape

(<tf.Tensor 'Shape_5:0' shape=(4,) dtype=int32>,
 TensorShape([Dimension(10), Dimension(299), Dimension(299), Dimension(3)]),
 TensorShape([Dimension(10), Dimension(299), Dimension(299), Dimension(3)]))

In [21]:
rank_three_tensor = tf.ones([3, 4, 5])
matrix = tf.reshape(rank_three_tensor, [6, 10])  # Reshape existing content into
                                                 # a 6x10 matrix
matrixB = tf.reshape(matrix, [3, -1])  #  Reshape existing content into a 3x20
                                       # matrix. -1 tells reshape to calculate
                                       # the size of this dimension.
matrixAlt = tf.reshape(matrixB, [4, 3, -1])  # Reshape existing content into a
                                             #4x3x5 tensor

# Note that the number of elements of the reshaped Tensors has to match the
# original number of elements. Therefore, the following example generates an
# error because no possible value for the last dimension will match the number
# of elements.
# yet_another = tf.reshape(matrixAlt, [13, 2, -1])  # ERROR!

It is not possible to have a tf.Tensor with more than one data type. It is possible, however, to **serialize arbitrary data structures as strings and store those in tf.Tensors**.

In [24]:
# Cast a constant integer tensor into floating point.
float_tensor = tf.cast(tf.constant([1, 2, 3]), dtype=tf.float32)

In [25]:
float_tensor.dtype

tf.float32

When creating a tf.Tensor from a python object you may optionally specify the datatype. If you don't, TensorFlow chooses a datatype that can represent your data. 

- The simplest way to evaluate a Tensor is using the `Tensor.eval` method
- The `eval` method only works when a default tf.Session is active
- `Tensor.eval` returns a numpy array with the same contents as the tensor.



In [37]:
with tf.Session() as sess:
    constant = tf.constant([1, 2, 3])
    tensor = constant * constant
    print(tensor.eval())
    
    p = tf.placeholder(tf.float32)
    t = p + 1.0
    # t.eval()  # This will fail, since the placeholder did not get a value.
    print(t.eval(feed_dict={p:2.0}))  # This will succeed because we're feeding a value
                               # to the placeholder.

[1 4 9]
3.0


Note that it is **possible to feed any tf.Tensor**, not just placeholders.

TensorFlow can't directly evaluate tf.Tensors defined inside functions or inside control flow constructs. If a tf.Tensor depends on a value from a queue, evaluating the tf.Tensor will only work once something has been enqueued; otherwise, evaluating it will hang. When working with queues, remember to call tf.train.start_queue_runners before evaluating any tf.Tensors.

In [40]:
t = tf.constant(12)
# This code prints the tf.Tensor object (which represents deferred computation) and not its value
print(t)

Tensor("Const_11:0", shape=(), dtype=int32)


- TensorFlow provides the tf.Print operation, which returns its first tensor argument unchanged while printing the set of tf.Tensors it is passed as the second argument.
- This is an identity op with the side effect of printing data when evaluating
- To correctly use tf.Print its return value must be used.

In [74]:
tf.Print(t, [t])  # This does nothing
t = tf.Variable(tf.random_normal([2, 2]))
t_print = tf.Print(t, [t])  # Here we are using the value returned by tf.Print
result = t_print + 1  # Now when result is evaluated the value of `t` will be printed.
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
print('result={}'.format(sess.run(result)))

result=[[-0.9177327  -0.07852054]
 [ 0.41223937 -0.24745262]]


In [70]:
import tensorflow as tf
from tensorflow.python import debug as tf_debug
a = tf.constant([1.0, 4.0], shape=[2,1])
b = tf.constant([2.0, 3.0], shape=[1,2])
c = tf.add(tf.matmul(a,b), tf.constant([5.0, 6.0]))
d = tf.Print(c, [c, 2.0], message="Value of C is:")
sess = tf_debug.LocalCLIDebugWrapperSession(sess)
with tf.Session() as sess:
    sess.run(d)